In [21]:
import scipy 
from scipy.optimize import minimize
import numpy as np
import math
import itertools
import time
from itertools import product
import pandas as pd
!pip install tqdm
from tqdm.notebook import trange,tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
def convert_lambdas_to_prices(lambdas, inverse_demand_func):
  prices = []
  for rate in lambdas:
    prices.append(float(inverse_demand_func(rate)))
  return prices

In [23]:
def compute_steady_state(lambdas, NUM_SERVERS):
  mus = [min(NUM_SERVERS,i) for i in range(len(lambdas)+1)]
  lambda_mu_ratios = [lambdas[i]/mus[i+1] for i in range(len(lambdas))]
  denominator = np.sum([np.product(lambda_mu_ratios[0:k]) for k in range(len(lambdas)+1)])
  steady_state = [np.product(lambda_mu_ratios[0:k])/denominator  for k in range(len(lambdas)+1)]
  return steady_state 

In [24]:
def construct_function(inverse_demand_func, NUM_SERVERS):
    
    def negative_func(lambdas):
        mus = [min(NUM_SERVERS,i) for i in range(len(lambdas)+1)]

        lambda_mu_ratios = [lambdas[i]/mus[i+1] for i in range(len(lambdas))]

        denominator = np.sum([np.product(lambda_mu_ratios[0:k]) for k in range(len(lambdas)+1)])

        numerator_sum_terms = []
        for i in range(len(lambdas)):
            expected_cost = 1/NUM_SERVERS * (i+1) if i >= NUM_SERVERS else 1

            if abs(lambdas[i])>1e-12:
              term = (inverse_demand_func(lambdas[i])-expected_cost) * lambdas[i] * np.product(lambda_mu_ratios[0:i])
            else:
              term = 0            
            numerator_sum_terms.append(term)                      

        return -1*np.sum(numerator_sum_terms)/denominator
    
    return negative_func

In [25]:
 def construct_static_function(inverse_demand_func, cutoff, NUM_SERVERS):
    
    def negative_func(arrival_rate):
        arrival_rate = arrival_rate[0]
        
        lambdas = [arrival_rate for i in range(cutoff+1)]

        mus = [min(NUM_SERVERS,i) for i in range(len(lambdas)+1)]

        lambda_mu_ratios = [lambdas[i]/mus[i+1] for i in range(len(lambdas))]

        denominator = np.sum([np.product(lambda_mu_ratios[0:k]) for k in range(len(lambdas)+1)])

        numerator_sum_terms = []
        for i in range(len(lambdas)):
            expected_cost = 1/NUM_SERVERS * (i+1) if i >= NUM_SERVERS else 1
            if lambdas[i]:
              term = (inverse_demand_func(lambdas[i])-expected_cost) * lambdas[i] * np.product(lambda_mu_ratios[0:i])
            else:
              term = 0
            numerator_sum_terms.append(term) 

        return -1*np.sum(numerator_sum_terms)/denominator
    
    return negative_func    

In [26]:
def compute_optimal_static_policy_given_rate(inverse_demand_func, MAX_QUEUE, arrival_rate, NUM_SERVERS):
  profit_vals = []
  for cutoff in range(MAX_QUEUE):
    static_func = construct_static_function(inverse_demand_func,cutoff, NUM_SERVERS)
    profit_vals.append(-1*static_func(arrival_rate))

  best_cutoff = np.argmax(profit_vals)
  
  return [arrival_rate[0] if i < best_cutoff+1 else 0 for i in range(MAX_QUEUE)]



In [27]:
def compute_optimal_static_policy(inverse_demand_func, MAX_QUEUE, NUM_SERVERS, MAX_RATE):
    max_so_far = -np.inf
    for cutoff in range(MAX_QUEUE):
        static_func = construct_static_function(inverse_demand_func,cutoff,NUM_SERVERS)
        bounds = [(0, MAX_RATE)]
        initial_sol = [0]
        res = minimize(static_func, initial_sol, bounds = bounds, options = {'maxfun': 30000})
        
        if -1*res.fun >= max_so_far:
          max_so_far = -1*res.fun
          best_rate = res.x
          best_cutoff = cutoff

    return [best_rate[0] if i < best_cutoff+1 else 0 for i in range(MAX_QUEUE)]

In [28]:
def compute_optimal_dynamic_policy(inverse_demand_func, MAX_QUEUE, NUM_SERVERS, initial_sol, MAX_RATE=1e20):
  negative_func = construct_function(inverse_demand_func, NUM_SERVERS)
  bounds = [(0, MAX_RATE) for i in range(MAX_QUEUE)]
  A = generate_constraint_matrix(MAX_QUEUE)
  cons = [{"type": "ineq", "fun": lambda x: A @ x}]
  res = minimize(negative_func, initial_sol, bounds = bounds, constraints = cons)
  return res.x, -1*res.fun

In [29]:
def get_rev_cost_profit_from_policy(policy, NUM_SERVERS, inverse_demand_func):
  steady_state = compute_steady_state(policy, NUM_SERVERS)
  pricing_policy = convert_lambdas_to_prices(policy, inverse_demand_func)
  rev = np.dot(steady_state[0:-1],np.multiply(pricing_policy,policy))
  cost = np.dot(steady_state, np.arange(len(steady_state)))
  profit = rev-cost

  return rev,cost,profit

In [30]:
def generate_constraint_matrix(n):
  A = []

  for i in range(n-1):
    row = np.zeros(n)
    row[i] = 1
    row[i+1] = -1
    A.append(row)
  return np.array(A)

In [ ]:
#Test Parameters
num_servers_to_test = 1 #Number of servers 
demand_type = 'linear' #demand_type can be 'logistic', 'linear', or 'exponential'
num_instances = 1000 #number of instances

test_vals = []
while len(test_vals) != num_instances:
    a = np.random.rand(1)*4.9+.1
    b = np.random.rand(1)*9.5+.5
    p0 = np.random.rand(1)*20
    test_vals.append([a,b, p0])


start_time = time.time()
experiment_results = []
elmachtoub_rev_ratios = []
elmachtoub_cost_ratios = []
elmachtoub_profit_ratios = []
ab_vals = []
blocking_probs = []
optimal_rev_ratios = []
optimal_cost_ratios = []
optimal_profit_ratios = []
num_servers_to_test = [num_servers_to_test]

degenerate_instances_counter = 0


for NUM_SERVERS in num_servers_to_test:
    this_server_experiments = []
    for i in trange(len(test_vals)):

      (a,b,p0) = test_vals[i]

      if demand_type == 'linear':
        MAX_QUEUE = math.ceil(b/a)*NUM_SERVERS+1
        inverse_demand_func = lambda x: (b-x)/a

      if demand_type == 'exponential':
        MAX_QUEUE = max(math.ceil(b/a)*3, NUM_SERVERS)
        MAX_QUEUE = 20
        inverse_demand_func = lambda x: math.log(b/x)/a if x !=0 else 1e40

      if demand_type == 'logistic':
        MAX_QUEUE = math.ceil(b/a)*5*NUM_SERVERS
        def inverse_logistic(x):
          if abs(x-b) <= 1e-5:
            return 0
          if x == 0:
            return 1e20
          else:
            try:
              return math.log(b*((1+math.exp(-a*p0))/x)-1)/a + p0
            except ValueError:
              print(x)
        inverse_demand_func = inverse_logistic

      MAX_RATE = b

      optimal_static_policy = compute_optimal_static_policy(inverse_demand_func, MAX_QUEUE, NUM_SERVERS, MAX_RATE)

      optimal_dynamic_policy, optimal_profit = compute_optimal_dynamic_policy(inverse_demand_func, MAX_QUEUE, NUM_SERVERS, initial_sol = optimal_static_policy, MAX_RATE=b)

      optimal_steady_state = compute_steady_state(optimal_dynamic_policy,NUM_SERVERS)

      blocking_probs.append(optimal_steady_state[-1])

      if optimal_steady_state[-1] > .001:
        print('Max queue needs to be higher')

      elmachtoub_rate = np.dot(optimal_steady_state[0:-1],optimal_dynamic_policy)
      
      optimal_elmachtoub_policy = compute_optimal_static_policy_given_rate(inverse_demand_func,MAX_QUEUE,[elmachtoub_rate], NUM_SERVERS)

      opt_rev,opt_cost,opt_profit = get_rev_cost_profit_from_policy(optimal_dynamic_policy, NUM_SERVERS, inverse_demand_func)

      opt_static_rev,opt_static_cost,opt_static_profit = get_rev_cost_profit_from_policy(optimal_static_policy, NUM_SERVERS, inverse_demand_func)

      opt_elmachtoub_rev,opt_elmachtoub_cost,opt_elmachtoub_profit = get_rev_cost_profit_from_policy(optimal_elmachtoub_policy, NUM_SERVERS, inverse_demand_func)

      if opt_profit != 0 and len(elmachtoub_rev_ratios) < 1000:
        elmachtoub_rev_ratios.append(opt_elmachtoub_rev/opt_rev)
        optimal_rev_ratios.append(opt_static_rev/opt_rev)

        elmachtoub_cost_ratios.append(opt_elmachtoub_cost/opt_cost)
        optimal_cost_ratios.append(opt_static_cost/opt_cost)

        elmachtoub_profit_ratios.append(opt_elmachtoub_profit/opt_profit)
        optimal_profit_ratios.append(opt_static_profit/opt_profit)
      else:
        degenerate_instances_counter+=1
        #elmachtoub_rev_ratios.append(1)
        #optimal_rev_ratios.append(1)

        #elmachtoub_cost_ratios.append(1)
        #optimal_cost_ratios.append(1)

        #elmachtoub_profit_ratios.append(1)
        #optimal_profit_ratios.append(1)
      ab_vals.append([a,b])


duration = time.time()-start_time

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [ ]:
np.round(np.mean(optimal_profit_ratios)*100,2), np.round(np.min(optimal_profit_ratios)*100,2)

In [ ]:
np.round(np.mean(elmachtoub_profit_ratios)*100,2), np.round(np.min(elmachtoub_profit_ratios)*100,2)

(99.94, 98.61)

In [ ]:
np.round(np.mean(optimal_rev_ratios)*100,2), np.round(np.min(optimal_rev_ratios)*100,2)

(99.99, 99.18)

In [ ]:
np.round(np.mean(elmachtoub_rev_ratios)*100,2), np.round(np.min(elmachtoub_rev_ratios)*100,2)

(99.96, 98.41)

In [ ]:
np.round(np.mean(optimal_cost_ratios)*100,2), np.round(np.max(optimal_cost_ratios)*100,2)

(100.24, 102.7)

In [ ]:
np.round(np.mean(elmachtoub_cost_ratios)*100,2), np.round(100*np.max(elmachtoub_cost_ratios))

(100.12, 102.0)